<a href="https://colab.research.google.com/github/robsmokos/Projekt_Zlozony/blob/main/PROJEKT_ZLOZONY_WSZIB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

PROJEKT ZLOZONY WSZIB 2024


In [1]:
pip install pycountry_convert

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 47.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 234.9/234.9 kB 12.9 MB/s eta 0:00:00


In [2]:
!pip install plotly

In [3]:
#BIBLIOTEKI
import pandas as pd
import numpy as np
import datetime as dt
import requests
import sys
from itertools import chain
import pycountry
import pycountry_convert as pc
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
%matplotlib inline

from sklearn.preprocessing import OrdinalEncoder
from sklearn import metrics
import xgboost as xgb
from xgboost import XGBRegressor
from xgboost import plot_importance, plot_tree
from sklearn.model_selection import GridSearchCV
import warnings
warnings.filterwarnings('ignore')

In [4]:
# Poprawione linki
df = pd.read_csv('https://drive.google.com/uc?id=1KdZTtrFYD-yFKimQWXCe0K26x-9F8YR-')
#df_train = pd.read_csv('https://drive.google.com/uc?id=1zgbhgOyuMFJbh1vbgxZapWlrO501PyEn')
df_test = pd.read_csv('https://drive.google.com/uc?id=14M8lojVb7YWqSaWvvuvTA1bR32uQ_XSd')




In [5]:
# Zmiana nazw kolumn w DataFrame dla lepszej czytelności
df.rename(columns={
    'ObservationDate': 'Date',  # Zmiana nazwy kolumny 'ObservationDate' na 'Date'
    'Province/State': 'Province_State',  # Zmiana nazwy kolumny 'Province/State' na 'Province_State'
    'Country/Region': 'Country_Region',  # Zmiana nazwy kolumny 'Country/Region' na 'Country_Region'
    'Confirmed': 'ConfirmedCases',  # Zmiana nazwy kolumny 'Confirmed' na 'ConfirmedCases'
    'Deaths': 'Fatalities'  # Zmiana nazwy kolumny 'Deaths' na 'Fatalities'
}, inplace=True)

# Zastąpienie wartości 'Mainland China' nazwą 'China' w kolumnie 'Country_Region'
df.loc[df['Country_Region'] == 'Mainland China', 'Country_Region'] = 'China'

# Konwersja kolumny 'Date' na format daty
df['Date'] = pd.to_datetime(df['Date'], format='%m/%d/%Y')  # Przekształcenie tekstu w kolumnie 'Date' na daty

# Dodanie kolumny 'Day', która wskazuje numer dnia w roku
df['Day'] = df.Date.dt.dayofyear  # Ekstrakcja numeru dnia roku z kolumny 'Date'

# Tworzenie kolumny 'cases_lag_1', która zawiera dane o przypadkach z poprzedniego dnia (lag 1)
df['cases_lag_1'] = df.groupby(['Country_Region', 'Province_State'])['ConfirmedCases'].shift(1)

# Tworzenie kolumny 'deaths_lag_1', która zawiera dane o zgonach z poprzedniego dnia (lag 1)
df['deaths_lag_1'] = df.groupby(['Country_Region', 'Province_State'])['Fatalities'].shift(1)

# Obliczanie dziennej liczby nowych przypadków
df['Daily Cases'] = df['ConfirmedCases'] - df['cases_lag_1']  # Liczba nowych przypadków to różnica między dzisiejszymi a wczorajszymi

# Obliczanie dziennej liczby nowych zgonów
df['Daily Deaths'] = df['Fatalities'] - df['deaths_lag_1']  # Liczba nowych zgonów to różnica między dzisiejszymi a wczorajszymi


In [85]:


#display(df_train.head())
#display(df_train.describe())
#display(df_train.info())





,Id,Province_State,Country_Region,Date,ConfirmedCases,Fatalities
0,1,NaN,Afghanistan,2020-01-22,0.0,0.0
1,2,NaN,Afghanistan,2020-01-23,0.0,0.0
2,3,NaN,Afghanistan,2020-01-24,0.0,0.0
3,4,NaN,Afghanistan,2020-01-25,0.0,0.0
4,5,NaN,Afghanistan,2020-01-26,0.0,0.0


,Id,Date,ConfirmedCases,Fatalities
count,35995.000000,35995,35995.000000,35995.000000
mean,17998.000000,2020-03-19 00:00:00.000000256,3683.508737,243.560217
min,1.000000,2020-01-22 00:00:00,0.000000,0.000000
25%,8999.500000,2020-02-19 00:00:00,0.000000,0.000000
50%,17998.000000,2020-03-19 00:00:00,19.000000,0.000000
75%,26996.500000,2020-04-17 00:00:00,543.000000,7.000000
max,35995.000000,2020-05-15 00:00:00,345813.000000,33998.000000
std,10391.005806,NaN,18986.978708,1832.966999


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35995 entries, 0 to 35994
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Id              35995 non-null  int64         
 1   Province_State  15295 non-null  object        
 2   Country_Region  35995 non-null  object        
 3   Date            35995 non-null  datetime64[ns]
 4   ConfirmedCases  35995 non-null  float64       
 5   Fatalities      35995 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(1), object(2)
memory usage: 1.6+ MB


None

In [ ]:


# Konwersja kolumny 'Date' w zbiorze df_train na format daty (%Y-%m-%d), np. 2021-12-31
##df_train['Date'] = pd.to_datetime(df_train['Date'], format='%Y-%m-%d')

# Konwersja kolumny 'Date' w zbiorze df_test na format daty (%Y-%m-%d), np. 2021-12-31
##df_test['Date'] = pd.to_datetime(df_test['Date'], format='%Y-%m-%d')




In [ ]:


##train_date_min = df_train['Date'].min()
##train_date_max = df_train['Date'].max()
##print('Minimum date from training set: {}'.format(train_date_min))
##print('Maximum date from training set: {}'.format(train_date_max))



Minimum date from training set: 2020-01-22 00:00:00
Maximum date from training set: 2020-05-15 00:00:00


In [ ]:


##test_date_min = df_test['Date'].min()
##test_date_max = df_test['Date'].max()
##print('Minimum date from test set: {}'.format(test_date_min))
##print('Maximum date from test set: {}'.format(test_date_max))



Minimum date from test set: 2020-04-02 00:00:00
Maximum date from test set: 2020-05-14 00:00:00


In [6]:
class country_utils():
    def __init__(self):
        self.d = {}

    def get_dic(self):
        return self.d

    def get_country_details(self,country):
        """Returns country code(alpha_3) and continent"""
        try:
            country_obj = pycountry.countries.get(name=country)
            if country_obj is None:
                c = pycountry.countries.search_fuzzy(country)
                country_obj = c[0]
            continent_code = pc.country_alpha2_to_continent_code(country_obj.alpha_2)
            continent = pc.convert_continent_code_to_continent_name(continent_code)
            return country_obj.alpha_3, continent
        except:
            if 'Congo' in country:
                country = 'Congo'
            elif country == 'Diamond Princess' or country == 'Laos' or country == 'MS Zaandam'\
            or country == 'Holy See' or country == 'Timor-Leste':
                return country, country
            elif country == 'Korea, South' or country == 'South Korea':
                country = 'Korea, Republic of'
            elif country == 'Taiwan*':
                country = 'Taiwan'
            elif country == 'Burma':
                country = 'Myanmar'
            elif country == 'West Bank and Gaza':
                country = 'Gaza'
            else:
                return country, country
            country_obj = pycountry.countries.search_fuzzy(country)
            continent_code = pc.country_alpha2_to_continent_code(country_obj[0].alpha_2)
            continent = pc.convert_continent_code_to_continent_name(continent_code)
            return country_obj[0].alpha_3, continent

    def get_iso3(self, country):
        return self.d[country]['code']

    def get_continent(self,country):
        return self.d[country]['continent']

    def add_values(self,country):
        self.d[country] = {}
        self.d[country]['code'],self.d[country]['continent'] = self.get_country_details(country)

    def fetch_iso3(self,country):
        if country in self.d.keys():
            return self.get_iso3(country)
        else:
            self.add_values(country)
            return self.get_iso3(country)

    def fetch_continent(self,country):
        if country in self.d.keys():
            return self.get_continent(country)
        else:
            self.add_values(country)
            return self.get_continent(country)

In [7]:


df.ConfirmedCases = np.abs(df.ConfirmedCases)
df_tm = df.copy()
date = df_tm.Date.max()#get current date
df_tm = df_tm[df_tm['Date']==date]
obj = country_utils()
df_tm.Province_State.fillna('',inplace=True)
df_tm['continent'] = df_tm.apply(lambda x: obj.fetch_continent(x['Country_Region']), axis=1)
df_tm["world"] = "World" # in order to have a single root node
fig = px.treemap(df_tm, path=['world', 'continent', 'Country_Region'], values='ConfirmedCases',
                  color='ConfirmedCases', hover_data=['Country_Region'],
                  color_continuous_scale='dense', title='Aktualny udział przypadków COVID-19 na świecie')
fig.update_layout(width=700,template='seaborn')
fig.show()

In [ ]:
# Sprawdźmy, czy w kolumnie 'ConfirmedCases' nie ma brakujących wartości, nieskończoności lub ujemnych liczb
df.ConfirmedCases = df.ConfirmedCases.replace([np.inf, -np.inf], np.nan)  # Zastępujemy nieskończoności NaN
df.ConfirmedCases.fillna(0, inplace=True)  # Zastępujemy brakujące wartości zerami

# Upewniamy się, że wszystkie wartości są dodatnie (bo liczba przypadków nie może być ujemna)
df.ConfirmedCases = np.abs(df.ConfirmedCases)

# Tworzymy kopię danych
df_tm = df.copy()

# Pobieramy aktualną datę
date = df_tm.Date.max()

# Filtrujemy dane tylko dla Polski i na podstawie aktualnej daty
df_tm = df_tm[(df_tm['Date'] == date) & (df_tm['Country_Region'] == 'Poland')]

# Inicjalizacja obiektu do uzyskiwania kontynentu na podstawie kraju
obj = country_utils()

# Wypełnianie brakujących wartości w kolumnie Province_State pustym ciągiem
df_tm.Province_State.fillna('', inplace=True)

# Dodanie kolumny kontynentu na podstawie kraju
df_tm['kontynent'] = df_tm.apply(lambda x: obj.fetch_continent(x['Country_Region']), axis=1)

# Dodanie wspólnego węzła "Świat"
df_tm["świat"] = "Świat"

# Tworzymy wykres drzewa tylko dla Polski
fig = px.treemap(df_tm, path=['świat', 'kontynent', 'Country_Region'], values='ConfirmedCases',
                  color='ConfirmedCases', hover_data=['Country_Region'],
                  color_continuous_scale='dense', title='Aktualny udział przypadków COVID-19 w Polsce')

# Dostosowanie wyglądu wykresu
fig.update_layout(width=700, template='seaborn')

# Wyświetlenie wykresu
fig.show()
